In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

In [3]:
def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [4]:
throttle = 0.7

def move_forward(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

def stop(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0
    control.brake = 1.0
    control.hand_brake = 1.0
    vehicle.apply_control(control)
 

In [30]:
def calculate_opposite_side(angle_radiant, side_lenght):
    return abs(side_lenght * math.tan(angle_radiant))

In [48]:
try:
    vehicle = spawn_vehicle()
    move_forward(vehicle)

    rad_cam = None
    rad_bp = world.get_blueprint_library().find('sensor.other.radar')
    rad_bp.set_attribute('horizontal_fov', str(35))
    rad_bp.set_attribute('vertical_fov', str(20))
    rad_bp.set_attribute('range', str(250))
    rad_location = carla.Location(x=2.0, z=1.0)
    rad_rotation = carla.Rotation()
    rad_transform = carla.Transform(rad_location,rad_rotation)
    rad_ego = world.spawn_actor(rad_bp,rad_transform,attach_to=vehicle, attachment_type=carla.AttachmentType.Rigid)

    def rad_callback(radar_data):
        for detect in radar_data:
            if calculate_opposite_side(detect.altitude, detect.depth) < 0.4 and calculate_opposite_side(detect.azimuth, detect.depth) < 0.9 and detect.velocity < 0 and detect.depth < 25:  
                stop(vehicle)
                print(calculate_opposite_side(detect.altitude, detect.depth), calculate_opposite_side(detect.azimuth, detect.depth), detect.depth, detect.velocity)

    rad_ego.listen(lambda radar_data: rad_callback(radar_data))
    time.sleep(20)

finally:
    vehicle.destroy()
    

0.30642557935665343 0.7283149551329499 24.65556526184082 -18.54226303100586
0.3810022907645281 0.26442309764571087 22.116249084472656 -18.344797134399414
0.11541418815086006 0.12663830688157143 15.625126838684082 -13.62814712524414
0.2724877129800019 0.08059157106494491 15.630677223205566 -13.648493766784668
0.006490814193918233 0.6784343439935694 15.637667655944824 -13.601049423217773
0.11106696896082491 0.867316989341957 15.644821166992188 -13.576335906982422
0.19376726747467218 0.6659420379394746 15.640491485595703 -13.626448631286621
0.04280406793982092 0.25789629712468765 15.624373435974121 -13.616987228393555
0.25598776713062593 0.6398232099399589 15.644139289855957 -13.635224342346191
0.04766887020086446 0.02322429578695166 15.040841102600098 -13.281647682189941
0.005540211343427815 0.18396578105262992 15.039582252502441 -13.278915405273438
0.15928151814731634 0.21335721514246975 15.035042762756348 -13.27169418334961
0.2073085259228313 0.1807130666212735 15.03348445892334 -13.26